<a href="https://colab.research.google.com/github/avinashjairam/ML_FINAL_PROJECT/blob/main/Goswami_Catto_Jairam_ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Much of this code adapted from github user ZFTurbo:

https://github.com/ZFTurbo/Keras-Mask-RCNN-for-Open-Images-2019-Instance-Segmentation/blob/master/data_segmentation/challenge-2019-classes-description-segmentable.csv

In [12]:
import os
#os.chdir('')
import sys
import csv
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf


tf.compat.v1.disable_eager_execution()

# Root directory of the project
#ROOT_DIR = os.path.abspath(".")

# Import Mask RCNN
#sys.path.append(ROOT_DIR)  # To find local version of the library
from mask_rcnn.mrcnn import utils
import mask_rcnn.mrcnn.model as modellib
from mask_rcnn.mrcnn import visualize
# Import COCO config
#sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
from mask_rcnn.samples.coco import coco

%matplotlib inline 

# Directory to save logs and trained model
#MODEL_DIR = os.path.join(ROOT_DIR, "logs")
MODEL_DIR = 'mask_rcnn/logs/'
# Local path to trained weights file
#COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
COCO_MODEL_PATH = 'mask_rcnn/mask_rcnn_coco.h5'
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = "images/"

In [2]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [3]:
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

Instructions for updating:
Use `tf.cast` instead.


In [31]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
coco_class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

oid_class_names = [
    'Clothing',
 'Girl',
 'Car',
 'Wheel',
 'Person',
 'Human body',
 'Woman',
 'Man',
 'Suit',
 'Flower',
 'Boy',
 'Skyscraper',
 'Jeans',
 'Dress',
 'Toy',
 'Bicycle wheel',
 'Sculpture',
 'Bottle',
 'Guitar',
 'Bird',
 'Dog',
 'Flag',
 'Drink',
 'Airplane',
 'Flowerpot',
 'Shorts',
 'Fish',
 'Book',
 'Motorcycle',
 'Cat',
 'Truck',
 'Hat',
 'Human mouth',
 'Duck',
 'Train',
 'Picture frame',
 'Horse',
 'Tie',
 'Bus',
 'Laptop',
 'Cattle',
 'Swimwear',
 'Trousers',
 'Shirt',
 'Balloon',
 'Van',
 'Goose',
 'Beer',
 'Rose',
 'Sun hat',
 'Wine',
 'Strawberry',
 'Cake',
 'Camera',
 'Mobile phone',
 'Human ear',
 'Coffee cup',
 'Common sunflower',
 'Tomato',
 'Box',
 'Cocktail',
 'Traffic sign',
 'Couch',
 'Computer keyboard',
 'Orange',
 'Pumpkin',
 'Canoe',
 'Muffin',
 'Bowl',
 'Pillow',
 'Christmas tree',
 'Taxi',
 'Fedora',
 'Ball',
 'Bread',
 'Vehicle registration plate',
 'Swan',
 'Platter',
 'Candle',
 'Football',
 'Roller skates',
 'Pastry',
 'Boot',
 'Mushroom',
 'Monkey',
 'Bronze sculpture',
 'Cowboy hat', 'High heels',
 'Cookie',
 'Juice',
 'Saucer',
 'Traffic light',
 'Elephant',
 'Handbag',
 'Penguin',
 'Chicken',
 'Carnivore',
 'Coin',
 'Vase',
 'Scarf']

oid_class_names = [x.lower() for x in oid_class_names]

x = set.intersection(set(coco_class_names), set(oid_class_names))
print(x, len(x))


{'elephant', 'airplane', 'orange', 'laptop', 'person', 'bus', 'train', 'tie', 'cat', 'bowl', 'bottle', 'cake', 'truck', 'motorcycle', 'vase', 'dog', 'car', 'bird', 'traffic light', 'horse', 'couch', 'book', 'handbag'} 23


In [26]:
file_names = list(os.listdir(IMAGE_DIR))
results_list = []
print(file_names)
for f in file_names:
    image = skimage.io.imread('images/' + f)
    results = model.detect([image], verbose=1)
    r = results[0]
    #print(type(results))
    #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            #class_names, r['scores'])

['puppies.jpg', 'light-switch.png', 'fruit.jpg']
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max:  248.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  143.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  int64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  int64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (512, 512, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1

## Training / Fine-tuning

We will be using the Mask R-CNN weights pretrained on the Coco dataset. In this project, we will be fine-tuning this model on a subset of the Open Images V6 dataset for the purpose of doing instance segmentation.

steps:

-- cut off top
-- freeze model weights
-- re-build top
-- 

In [27]:
from PIL import Image
def preprocess(image_set):
    # should return list of images all of the same size
    pass


def fine_tune(model, train_csv_file, num_classes):
    # train_csv_file is two columns: raw image | mask
    img_list = []
    mask_list = []
    with open(train_csv_file, 'r') as f:
        for line in f.readlines():
            line = line.split(',')
            img_file, mask_file = line[0], line[1]
            img = np.array(Image.open(img_file))
            mask = None
    img_size = x_train[0].shape[0]
    inputs = tf.keras.layers.Input(shape=(img_size, img_size, 3))
    top_dropout_rate = 0.2
    model.fit(x_train, y_train, epochs=16, callbacks=[
        tf.keras.callbacks.ModelCheckpoint('fine_tuned_mask_rcnn_oid.h5', verbose=1, save_best_model=True)
    ])
    return model
    



In [28]:
fine_tuned = fine_tune(model, )


TypeError: fine_tune() missing 3 required positional arguments: 'x_train', 'y_train', and 'num_classes'

## Testing / Evaluation

In [ ]:
def evaluate_model(model, x_test, y_test):
    pass
